In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load Excel file
file_path = 'ResCHNSeptoJan.xlsx'  # Update the path if needed
sheets = pd.ExcelFile(file_path).sheet_names

# Create output directory for plots
output_dir = 'EDA_Output'
os.makedirs(output_dir, exist_ok=True)

# Function to perform EDA on each sheet
def perform_eda(sheet_name):
    print(f"Processing sheet: {sheet_name}")
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Basic Information
    print(f"Sheet: {sheet_name}\n{df.info()}\n")
    
    # Identify columns with actual data (excluding 'No data')
    valid_columns = df.columns[~df.isin(['No Data']).all()]
    print(f"Columns with actual data in {sheet_name}: {list(valid_columns)}")
    with open(os.path.join(output_dir, f'{sheet_name}_valid_columns.txt'), 'w') as f:
        f.write("\n".join(valid_columns))
    
    # Summary Statistics
    summary = df.describe()
    summary.to_csv(os.path.join(output_dir, f'{sheet_name}_summary.csv'))
    
    # Missing Values
    missing_values = df.isnull().sum()
    missing_values.to_csv(os.path.join(output_dir, f'{sheet_name}_missing_values.csv'))
    
    # Distribution Analysis
    num_cols = df.select_dtypes(include=[np.number]).columns
    cat_cols = df.select_dtypes(exclude=[np.number]).columns
    
    for col in num_cols:
        plt.figure(figsize=(8, 5))
        sns.histplot(df[col].dropna(), kde=True, bins=30)
        plt.title(f'Distribution of {col}')
        plt.savefig(os.path.join(output_dir, f'{sheet_name}_{col}_distribution.png'))
        plt.close()
    
    for col in cat_cols:
        plt.figure(figsize=(8, 5))
        sns.countplot(y=df[col])
        plt.title(f'Count of {col}')
        plt.savefig(os.path.join(output_dir, f'{sheet_name}_{col}_countplot.png'))
        plt.close()
    
    # Outlier Detection (Boxplots)
    for col in num_cols:
        if df[col].nunique() > 1:
            plt.figure(figsize=(8, 5))
            sns.boxplot(x=df[col])
            plt.title(f'Boxplot of {col}')
            plt.savefig(os.path.join(output_dir, f'{sheet_name}_{col}_boxplot.png'))
            plt.close()
        else:
            print(f"Skipping boxplot for {col} (constant or empty)")
    
    # Correlation Matrix
    if len(num_cols) > 1:
        plt.figure(figsize=(10, 6))
        sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
        plt.title(f'Correlation Matrix - {sheet_name}')
        plt.savefig(os.path.join(output_dir, f'{sheet_name}_correlation_matrix.png'))
        plt.close()
    
    print(f"EDA for {sheet_name} completed. Outputs saved in {output_dir}\n")

# Run EDA on all sheets
for sheet in sheets:
    perform_eda(sheet)

print("EDA Completed for all sheets.")

Processing sheet: AHU
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8774 entries, 0 to 8773
Data columns (total 71 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Sno                            8774 non-null   int64  
 1   Machine Id                     8774 non-null   object 
 2   Machine Name                   8774 non-null   object 
 3   Group Name                     8774 non-null   object 
 4   Device Id                      8774 non-null   object 
 5   Timestamp                      8774 non-null   object 
 6   AHU_RD_AIE1-Th1_ChW_Fb_V-Vol   8774 non-null   object 
 7   AHU_RD_AIE2-RAT-DegC           8774 non-null   object 
 8   AHU_RD_AIE3-ChW_Val-Fb         8774 non-null   object 
 9   AHU_RD_DIE1-VFDTrip-Tr         8774 non-null   object 
 10  AHU_RD_DIE2-ChwValAuto-St      8774 non-null   object 
 11  AHU_RD_DIE2-ChwValMan-St       8774 non-null   object 
 12  AHU_RD_DIE2-VFDAM-St      

In [17]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Load Excel file
file_path = 'ResCHNSeptoJan.xlsx'  # Update the path if needed

# Automatically get the first sheet name
sheet_name = pd.ExcelFile(file_path).sheet_names[0]
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Split the dataframe based on 'Machine Name' column
machines = df['Machine Name'].unique()

df_splits = {machine:df[df['Machine Name'] == machine] for machine in machines}

# Save each split dataframe as a separate CSV file
output_dir = 'Split_Data'
os.makedirs(output_dir, exist_ok=True)

for machine, split_df in df_splits.items():
    split_df.to_csv(os.path.join(output_dir, f'{machine}.csv'), index=False)

# Plot distributions for each split dataset
plot_dir = 'Plots'
os.makedirs(plot_dir, exist_ok=True)

for machine, split_df in df_splits.items():
    numeric_cols = split_df.select_dtypes(include=['number']).columns
    
    for col in numeric_cols:
        plt.figure(figsize=(8, 5))
        sns.histplot(split_df[col].dropna(), kde=True, bins=30)
        plt.title(f'Distribution of {col} - {machine}')
        plt.savefig(os.path.join(plot_dir, f'{machine}_{col}_distribution.png'))
        plt.close()

print(f"Data has been split into {len(df_splits)} files based on Machine Name)
print("Distribution plots have been saved.")


Data has been split into 3 files based on Machine Name, with columns having significant 'No Data' removed.
Distribution plots have been saved.
